In [5]:
from fhd import *
import OpenEphys
import matplotlib.pyplot as plt
import numpy as np
from kaveh.toolbox import find_file
import re

In [2]:
source_path = '../data/behavioral/2019_09_12/'

In [6]:
fnames_behavioral = []
for root, dirnames, filenames in os.walk(source_path):
    for filename in filenames:
        f_regex = re.compile(r".*\.fhd$")
        if f_regex.match(filename):
            fnames_behavioral = fnames_behavioral + [os.path.join(root, filename)]
fnames_behavioral = np.array(f_names)

In [ ]:
fnames_

In [ ]:
for fn in fnames_behavioral:
    print('Loading {} ...'.format(fn))
    fhd_content = load(fn)

    HT = np.squeeze(fhd_content['target_x'])
    VT = np.squeeze(fhd_content['target_y'])
    HE = np.squeeze(fhd_content['right_horizontal_eye'])
    VE = np.squeeze(fhd_content['right_vertical_eye'])
    eye = np.linalg.norm(np.vstack((HE, VE)), axis = 0)
    # target = np.linalg.norm(np.vstack((target_x, target_y)), axis = 0)
    el_t = np.squeeze(fhd_content['t'])
    trials = fhd_content['trials']
    
    # Loading state times from behavioral data
    class trial_events:
        pass

    trialevents = []
    for trial in trials:
        trial_event = trial_events()
    #     if not np.array_equal(trial['state_start_time_cue_target_present'], trial['state_start_time_detect_sac_start']):
    #         print(trial['state_start_time_cue_target_present'], trial['state_start_time_detect_sac_start'])

        for k,v in zip(trial.keys(), trial.values()):
            exec('trial_event.' + k + '=v')
        trialevents.append(trial_event)
    